In [1]:
import pytz
import os
import sqlite3
import re
import uproot
import IPython
import sys
import copy
import numpy as np
import pandas as pd
import pymap3d as pm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from pandasql import sqldf
from rnog_data.runtable import RunTable
from datetime import datetime, timedelta
from scipy.fft import fft
from scipy.signal import stft
from NuRadioReco.modules.io.RNO_G.readRNOGDataMattak import readRNOGData
from NuRadioReco.utilities import units
from Flight import Flight
from IPython.display import clear_output

#-------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------
from FlightTracker import FlightTracker

current_month = '05'
next_month = '06'

start_time=f'2023-{current_month}-01 00:00:00'
stop_time=f'2023-{next_month}-01 00:00:00'
tracker_initial = FlightTracker(start_time, stop_time, destination='./flights/flights_2023.db', already_calculated=True)

In [2]:
tracker = copy.deepcopy(tracker_initial) # make a new instance of the tracker
print(len(tracker.flights), len(tracker.flights_distinct))
tracker.flights_distinct

103674 608


,index,flightnumber,date,filename,mintime,maxtime,min_r,min_z,theta
0,5441,N/A,2023-05-01,2023.05.02-051347.db,2023-05-01 08:05:52.100000,2023-05-01 20:09:18.100000,1.1,0.0,41.0
1,5442,PIA797,2023-05-01,2023.05.02-051347.db,2023-05-01 08:06:37.300000,2023-05-01 08:24:47.600000,34.7,8.6,29.0
2,5443,SAS936,2023-05-01,2023.05.02-051347.db,2023-05-01 10:28:08.600000,2023-05-01 10:47:04.100000,30.9,9.5,16.0
3,5444,ANA223,2023-05-01,2023.05.02-051347.db,2023-05-01 10:40:27.200000,2023-05-01 10:59:47.800000,12.5,10.2,40.0
4,5445,SAS931,2023-05-01,2023.05.02-051347.db,2023-05-01 11:20:49.000000,2023-05-01 11:37:33.800000,68.9,9.9,10.0
...,...,...,...,...,...,...,...,...,...
603,1703,AIC103,2023-05-31,2023.06.01-051619.db,2023-05-31 05:26:06.200000,2023-05-31 05:38:04.700000,122.8,10.4,54.0
604,1704,N/A,2023-05-31,2023.06.01-051619.db,2023-05-31 05:28:47.000000,2023-05-31 09:46:47.300000,77.2,8.8,67.0
605,1705,QTR65R,2023-05-31,2023.06.01-051619.db,2023-05-31 05:39:35.100000,2023-05-31 05:54:49.600000,72.3,8.3,6.0
606,1706,UAE41P,2023-05-31,2023.06.01-051619.db,2023-05-31 06:37:17.000000,2023-05-31 06:52:21.400000,72.4,8.9,6.0


In [3]:
# add fk_date_flightnumber
flights_distinct = tracker.flights_distinct
flights_distinct = sqldf(''' 
SELECT
    *
    ,date || '_' || flightnumber as fk_date_flightnumber
FROM flights_distinct
''')
flights = tracker.flights
flights = sqldf(''' 
SELECT
    *
    ,date(readtime) || '_' || flightnumber as fk_date_flightnumber
FROM flights
''')


In [4]:
# get header information for flight durations
dates_to_exclude_from_analysis_due_to_missing_detector_data = []

temp_df = pd.DataFrame()
length = len(flights_distinct)
for i in range(length):
    clear_output(wait=True)
    f = tracker.get_flight_by_index(i, filetype = 'headers.root')
    if len(f.header_df) > 0:
        temp_df = pd.concat([temp_df, f.header_df], ignore_index=True, sort=False)
    else: 
        print(f'No detector data for {f.date}')
        dates_to_exclude_from_analysis_due_to_missing_detector_data.append(f.date)
    
    print(f'end: {i + 1}/{length}')
    #print(f.header_df)

# filter header information for days where detector was on
flights_distinct = flights_distinct[~flights_distinct['date'].isin(dates_to_exclude_from_analysis_due_to_missing_detector_data)]#.reset_index()
flights = flights[~flights['date'].isin(dates_to_exclude_from_analysis_due_to_missing_detector_data)]#.reset_index()

print('ready to join tables')


end: 608/608
ready to join tables


In [5]:
print(len(temp_df))

9112756


In [6]:
#save the flights_distinct table with header information joined
con = sqlite3.connect('./header_files_for_time_range/header_2023')

# Write the DataFrame to the SQLite database
temp_df.to_sql(f'all_triggers_2023_{current_month}', con, if_exists = 'replace')

# Close the database connection
con.close()

In [7]:
#save the flights_distinct table with header information joined
con = sqlite3.connect('./header_files_for_time_range/header_2023')

# Write the DataFrame to the SQLite database
text = pd.read_sql_query(f'Select name from sqlite_master where type = "table"', con)

# Close the database connection
con.close()
text

,name
0,all_triggers_2023_03
1,all_triggers_2023_04
2,all_triggers_2023_06
3,all_triggers_2023_07
4,all_triggers_2023_08
5,all_triggers_2023_09
6,all_triggers_2023_10
7,all_triggers_2023_11
8,all_triggers_2023_05


In [5]:
con = sqlite3.connect('./header_files_for_time_range/header_2023')

temp_df = pd.read_sql_query(f'Select * from all_triggers_2023_05 where lt_triggers = 1', con)

con.close()
print(len(temp_df))

4856109


In [6]:
flights_distinct['mintime'] = pd.to_datetime(flights_distinct.mintime).dt.tz_localize('UTC')
flights_distinct['maxtime'] = pd.to_datetime(flights_distinct.maxtime).dt.tz_localize('UTC')

flights_distinct['mintime_millis'] = flights_distinct.maxtime.astype(np.int64) // 10 ** 6
flights_distinct['maxtime_millis'] = flights_distinct.maxtime.astype(np.int64) // 10 ** 6


temp_df['trigger_time_millis'] = (temp_df['trigger_time'] * (10 ** 3)).astype(np.int64)

flights_distinct


,index,flightnumber,date,filename,mintime,maxtime,min_r,min_z,theta,fk_date_flightnumber,mintime_millis,maxtime_millis
0,5441,N/A,2023-05-01,2023.05.02-051347.db,2023-05-01 08:05:52.100000+00:00,2023-05-01 20:09:18.100000+00:00,1.1,0.0,41.0,2023-05-01_N/A,1682971758100,1682971758100
1,5442,PIA797,2023-05-01,2023.05.02-051347.db,2023-05-01 08:06:37.300000+00:00,2023-05-01 08:24:47.600000+00:00,34.7,8.6,29.0,2023-05-01_PIA797,1682929487600,1682929487600
2,5443,SAS936,2023-05-01,2023.05.02-051347.db,2023-05-01 10:28:08.600000+00:00,2023-05-01 10:47:04.100000+00:00,30.9,9.5,16.0,2023-05-01_SAS936,1682938024100,1682938024100
3,5444,ANA223,2023-05-01,2023.05.02-051347.db,2023-05-01 10:40:27.200000+00:00,2023-05-01 10:59:47.800000+00:00,12.5,10.2,40.0,2023-05-01_ANA223,1682938787800,1682938787800
4,5445,SAS931,2023-05-01,2023.05.02-051347.db,2023-05-01 11:20:49+00:00,2023-05-01 11:37:33.800000+00:00,68.9,9.9,10.0,2023-05-01_SAS931,1682941053800,1682941053800
...,...,...,...,...,...,...,...,...,...,...,...,...
603,1703,AIC103,2023-05-31,2023.06.01-051619.db,2023-05-31 05:26:06.200000+00:00,2023-05-31 05:38:04.700000+00:00,122.8,10.4,54.0,2023-05-31_AIC103,1685511484700,1685511484700
604,1704,N/A,2023-05-31,2023.06.01-051619.db,2023-05-31 05:28:47+00:00,2023-05-31 09:46:47.300000+00:00,77.2,8.8,67.0,2023-05-31_N/A,1685526407300,1685526407300
605,1705,QTR65R,2023-05-31,2023.06.01-051619.db,2023-05-31 05:39:35.100000+00:00,2023-05-31 05:54:49.600000+00:00,72.3,8.3,6.0,2023-05-31_QTR65R,1685512489600,1685512489600
606,1706,UAE41P,2023-05-31,2023.06.01-051619.db,2023-05-31 06:37:17+00:00,2023-05-31 06:52:21.400000+00:00,72.4,8.9,6.0,2023-05-31_UAE41P,1685515941400,1685515941400


In [10]:
# join header information on flights_distinct
query = '''
SELECT 
    date || '_' || flightnumber as fk_date_flightnumber
    ,flightnumber
    ,station_number
    ,count(trigger_time_millis) as triggers_per_second
    ,trigger_time_sec
    --,mintime
    --,mintime_stamp
    --,maxtime
    --,maxtime_stamp
    --,date
FROM flights_distinct
LEFT JOIN ( 
            SELECT
                trigger_time_millis
                ,force_triggers
                ,station_number
                ,round(trigger_time_millis / 1000, 0) as trigger_time_sec
            FROM temp_df
)t0

ON trigger_time_millis > flights_distinct.mintime_millis
--AND trigger_time_millis < flights_distinct.maxtime_millis
WHERE trigger_time_sec is not null
and min_r < 50
AND force_triggers = 0
--AND lt_triggers = 1
--AND radiant_triggers = 1
GROUP BY flightnumber, station_number, trigger_time_sec
'''

result = sqldf(query)
clear_output(wait=True)
result

: 

In [6]:
pd.options.display.float_format = '{:20,.2f}'.format

In [7]:
temp_df.trigger_time_millis

0          1682928357386
1          1682928369010
2          1682928369503
3          1682928371716
4          1682928374912
               ...      
4856104    1685526800990
4856105    1685526801763
4856106    1685526802987
4856107    1685526803648
4856108    1685526803869
Name: trigger_time_millis, Length: 4856109, dtype: int64

In [9]:
flights_distinct.mintime_stamp

0      1682971758100
1      1682929487600
2      1682938024100
3      1682938787800
4      1682941053800
           ...      
603    1685511484700
604    1685526407300
605    1685512489600
606    1685515941400
607    1685526804200
Name: mintime_stamp, Length: 608, dtype: int64